In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [2]:
data = np.load('data.npz')
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']

In [3]:
class SDHDataset(Dataset):
    def __init__(self, data, label, transform=None, target_transform=None):
        self.label = label
        self.data = data
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        data = self.data[idx]
        label = self.label[idx]
        
        if self.transform:
            data = self.transform(data)
        if self.target_transform:
            label = self.target_transform(label)
        return data, label

In [4]:
func = lambda x: torch.tensor(x, dtype=torch.float)
train_dataset = SDHDataset(X_train, y_train, transform = func, target_transform = func)
test_dataset = SDHDataset(X_test, y_test, transform = func, target_transform = func)

In [5]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [6]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(20, 15),
            nn.ReLU(),
            nn.Linear(15, 10),
            nn.ReLU(),
            nn.Linear(10, 10),
            nn.ReLU(),
            nn.Linear(10, 1),
            # nn.Sigmoid()  
        )

    def forward(self, x):
        return self.linear_relu_stack(x)

In [8]:
model = NeuralNetwork().to(device)

In [9]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    model.train()
    for batch, (X, y) in enumerate(dataloader):

        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)


        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    model.eval()
    num_batches = len(dataloader)
    test_loss = 0
    total_rmse = 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y.view(-1, 1)).item()
            
            mse = torch.mean((pred - y.view(-1, 1))**2).item()
            rmse = mse**0.5
            total_rmse += rmse
            
    test_loss /= num_batches
    avg_rmse = total_rmse / num_batches
    print(f"Test Avg loss: {test_loss:>8f}")
    print(f"Average RMSE: {avg_rmse:>8f}\n")


In [11]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.443754  [   64/  747]
Test Avg loss: 0.366966
Average RMSE: 0.605674

Epoch 2
-------------------------------
loss: 0.306712  [   64/  747]
Test Avg loss: 0.256404
Average RMSE: 0.506157

Epoch 3
-------------------------------
loss: 0.213602  [   64/  747]
Test Avg loss: 0.156912
Average RMSE: 0.395915

Epoch 4
-------------------------------
loss: 0.151456  [   64/  747]


/home/edward/Documents/projects/2024Fall/CS4774/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/edward/Documents/projects/2024Fall/CS4774/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([43])) that is different to the input size (torch.Size([43, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test Avg loss: 0.079170
Average RMSE: 0.278554

Epoch 5
-------------------------------
loss: 0.068622  [   64/  747]
Test Avg loss: 0.044313
Average RMSE: 0.210482

Epoch 6
-------------------------------
loss: 0.066806  [   64/  747]
Test Avg loss: 0.030488
Average RMSE: 0.174477

Epoch 7
-------------------------------
loss: 0.045260  [   64/  747]
Test Avg loss: 0.024150
Average RMSE: 0.155214

Epoch 8
-------------------------------
loss: 0.038299  [   64/  747]
Test Avg loss: 0.023210
Average RMSE: 0.152141

Epoch 9
-------------------------------
loss: 0.036935  [   64/  747]
Test Avg loss: 0.022185
Average RMSE: 0.148526

Epoch 10
-------------------------------
loss: 0.030560  [   64/  747]
Test Avg loss: 0.021714
Average RMSE: 0.147117

Epoch 11
-------------------------------
loss: 0.029027  [   64/  747]
Test Avg loss: 0.021444
Average RMSE: 0.146421

Epoch 12
-------------------------------
loss: 0.026793  [   64/  747]
Test Avg loss: 0.021065
Average RMSE: 0.144734

Epoch

In [12]:
class RandomNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(20, 40),
            nn.ReLU(),
            nn.Linear(40, 1),
            nn.Sigmoid()
        )
        
        # Initialize weights with high variance for more random-like behavior
        for layer in self.modules():
            if isinstance(layer, nn.Linear):
                nn.init.normal_(layer.weight, mean=0, std=2.0)
                nn.init.normal_(layer.bias, mean=0, std=2.0)
    
    def forward(self, x):
        return self.linear_relu_stack(x)

# Create model
model = RandomNetwork()